In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/ST/stargan

Mounted at /content/drive
/content/drive/MyDrive/ST/stargan


In [ ]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import LambdaLR
import torch.optim as optim
import os
import csv
import random
import copy

seed = 2710
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:
config = {
    'realworld': {
        'dataset_name': 'realworld_128_3ch_4cl',
        'num_df_domains': 10,
        'num_dp_domains': 5,
        'num_classes': 4,
        'class_names': ['WAL', 'RUN', 'CLD', 'CLU'],
        'num_timesteps': 128,
        'num_channels': 3,
        'num_classes': 4,
    },
    'cwru': {
        'dataset_name': 'cwru_256_3ch_5cl',
        'num_df_domains': 4,
        'num_dp_domains': 4,
        'num_classes': 5,
        'class_names': ['IR', 'Ball', 'OR_centred', 'OR_orthogonal', 'OR_opposite'],
        'num_timesteps': 256,
        'num_channels': 3,
        'num_classes': 5,
    }
}

In [ ]:
def get_data(dataset, domains_set, src_class, domain=None):
    # Load configurations
    dataset_name = config[dataset]['dataset_name']
    class_idx = config[dataset]['class_names'].index(src_class)
    num_df_domains = config[dataset]['num_df_domains']

    try:
        with open(f'data/{dataset_name}.pkl', 'rb') as f:
            x, y, k = pickle.load(f)
        with open(f'data/{dataset_name}_fs.pkl', 'rb') as f:
            fs = pickle.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"Dataset files for {dataset} not found.")


    if domains_set == 'df':
        mask = (y != class_idx) & (k < num_df_domains) & (fs == 0)
    elif domains_set == 'dp':
        mask = (y != class_idx) & (k >= num_df_domains) & (fs == 0)
    elif domains_set == 'dpfs':
        mask = (y != class_idx) & (k >= num_df_domains) & (fs == 1)
    else:
        raise ValueError(f"Invalid domains set: {domains_set}")

    # Apply initial mask
    x, y, k = x[mask], y[mask], k[mask]

    # Additional domain filtering if specified
    if domain is not None:
        domain_mask = (k == domain)
        x, y, k = x[domain_mask], y[domain_mask], k[domain_mask]

    assert len(x) > 0, f"No data found"
    assert len(x) == len(y) == len(k), f"Data length mismatch"

    return x, y, k



def get_syn_data(dataset, src_class, domain=None, mode='ref', fs=False):
    # Load configurations
    class_names = config[dataset]['class_names']

    try:
        if fs:
            with open(f'data/{dataset}_syndata_{mode}_fs.pkl', 'rb') as f:
                syndata = pickle.load(f)
        else:
            with open(f'data/{dataset}_syndata_{mode}.pkl', 'rb') as f:
                syndata = pickle.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"Dataset files for {dataset} not found.")

    x = []
    y = []
    k = []

    for trg_class in class_names:
        if trg_class == src_class:
            continue
        x_, y_, k_ = syndata[(src_class, trg_class)]
        x.append(x_)
        y.append(y_)
        k.append(k_)

    x = np.concatenate(x, axis=0)
    y = np.concatenate(y, axis=0)
    k = np.concatenate(k, axis=0)

    if domain is not None:
        domain_mask = (k == domain)
        x, y, k = x[domain_mask], y[domain_mask], k[domain_mask]

    assert len(x) > 0, f"No data found"
    assert len(x) == len(y) == len(k), f"Data length mismatch"

    return x, y, k



class TSTRClassifier(nn.Module):
    def __init__(self, num_timesteps=128, num_channels=3, num_classes=5):
        super(TSTRClassifier, self).__init__()

        self.conv1 = nn.Conv1d(num_channels, 16, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(16)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(32)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(64)
        self.conv4 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn4 = nn.BatchNorm1d(128)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.25)

        self.fc_shared = nn.Linear(num_timesteps * 8, 100)

        self.fc_class = nn.Linear(100, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(x)
        x = self.relu(self.fc_shared(x))

        # Final output for class prediction
        class_outputs = self.fc_class(x)
        return class_outputs



def remap_labels(y):
    label_map = {clss: i for i, clss in enumerate(np.unique(y))}
    return np.array([label_map[clss] for clss in y])



def get_dataloader(x, y, shuffle=False):
    x = torch.tensor(x, dtype=torch.float32)
    y = remap_labels(y)
    y = torch.tensor(y, dtype=torch.long)

    # Determine dataset size and batch size
    dataset_size = len(x)
    if dataset_size <= 100:
        batch_size = dataset_size
    elif dataset_size <= 1000:
        batch_size = 16
    elif dataset_size <= 5000:
        batch_size = 32
    else:
        batch_size = 64

    dataset = TensorDataset(x, y)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    return loader



def evaluate_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    loss_fn = nn.CrossEntropyLoss()

    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = loss_fn(outputs, y_batch)
            total_loss += loss.item()

            _, predicted_labels = torch.max(outputs, 1)
            correct_predictions += (predicted_labels == y_batch).sum().item()
            total_predictions += len(y_batch)

    total_loss /= len(test_loader)
    accuracy = correct_predictions / total_predictions

    return accuracy, total_loss



def train_model(model, train_loader, val_loader, optimizer, num_epochs=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    loss_fn = nn.CrossEntropyLoss()

    loss_train = []
    loss_val = []
    accuracy_val = []
    best_model_state = None
    best_loss = np.inf
    best_accuracy = 0

    # Set up linear learning rate decay
    lambda_lr = lambda epoch: 1 - epoch / num_epochs
    scheduler = LambdaLR(optimizer, lr_lambda=lambda_lr)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        total_loss /= len(train_loader)
        loss_train.append(total_loss)

        # Update learning rate
        scheduler.step()

        val_accuracy, val_loss = evaluate_model(model, val_loader)
        if val_loss < best_loss:
            best_epoch = epoch
            best_accuracy = val_accuracy
            best_loss = val_loss
            best_model_state = model.state_dict().copy()

        loss_val.append(val_loss)
        accuracy_val.append(val_accuracy)

        current_lr = scheduler.get_last_lr()[0]
        if (epoch+1) % 10 == 0:
            print(f"\tEpoch {epoch + 1}/{num_epochs} - Train loss: {total_loss:.4f} - Val accuracy: {val_accuracy:.4f} - Val loss: {val_loss:.4f} - LR: {current_lr:.2e}")

    print(f"\tBest epoch: {best_epoch + 1} - Best val accuracy: {best_accuracy:.4f} - Best val loss: {best_loss:.4f}\n")

    # Load best model state
    model.load_state_dict(best_model_state)

    return model



def train_and_test(x_train, y_train, x_test, y_test, dataset, num_epochs=50):
    assert np.array_equal(np.unique(y_train), np.unique(y_test)), "Training and test labels do not match"

    x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, shuffle=True, random_state=seed)

    train_loader = get_dataloader(x_tr, y_tr, shuffle=True)
    val_loader = get_dataloader(x_val, y_val)
    test_loader = get_dataloader(x_test, y_test)

    model = TSTRClassifier(num_timesteps=config[dataset]['num_timesteps'],
                           num_channels=config[dataset]['num_channels'],
                           num_classes=config[dataset]['num_classes']-1)
    initial_lr = 0.0001
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)

    trained_model = train_model(model, train_loader, val_loader, optimizer, num_epochs)

    test_accuracy, test_loss = evaluate_model(trained_model, test_loader)

    return test_accuracy, test_loss



def train_only(x_train, y_train, dataset, num_epochs=50):

    x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, shuffle=True, random_state=seed)

    train_loader = get_dataloader(x_tr, y_tr, shuffle=True)
    val_loader = get_dataloader(x_val, y_val)

    model = TSTRClassifier(num_timesteps=config[dataset]['num_timesteps'],
                           num_channels=config[dataset]['num_channels'],
                           num_classes=config[dataset]['num_classes']-1)
    initial_lr = 0.0001
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)

    trained_model = train_model(model, train_loader, val_loader, optimizer, num_epochs)

    return trained_model



def train_classifier_cv(x_train, y_train, dataset, num_epochs=50):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

    accs = []
    losses = []

    for train_index, test_index in skf.split(x_train, y_train):
        x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]
        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

        acc, loss = train_and_test(x_train_fold, y_train_fold, x_test_fold, y_test_fold, dataset, num_epochs)
        accs.append(acc)
        losses.append(loss)

    return np.mean(accs), np.mean(losses)



def fine_tune(model, x_train, y_train, num_epochs=50):
    x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, shuffle=True, random_state=seed)

    train_loader = get_dataloader(x_tr, y_tr, shuffle=True)
    val_loader = get_dataloader(x_val, y_val)

    initial_lr = 0.00001
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)

    trained_model = train_model(model, train_loader, val_loader, optimizer, num_epochs)

    return trained_model



def save_scores(source, domain, mode, accuracy, loss, name, dataset):
    results_dir = 'results'
    # Ensure the directory exists
    os.makedirs(results_dir, exist_ok=True)
    # Path to the CSV file
    file_path = os.path.join(results_dir, f'{dataset}_{name}.csv')
    # Check if the file exists
    file_exists = os.path.exists(file_path)

    # Open the file in append mode if it exists, or write mode if it doesn't
    with open(file_path, mode='a' if file_exists else 'w', newline='') as file:
        writer = csv.writer(file)
        # If the file does not exist, write the header
        if not file_exists:
            writer.writerow(['source', 'domain', 'mode', 'accuracy', 'loss'])
        # Write the data rows
        writer.writerow([source, domain, mode, accuracy, loss])

In [ ]:
def compute_TSTR_Dp(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train and evaluate via cross-validation on Dp data
            print('Training and evaluating on Dp data via cross-validation...')
            acc, loss = train_classifier_cv(x_dp_dom, y_dp_dom, dataset, num_epochs=50)
            save_scores(src_class, domain, 'real', acc, loss, 'Dp', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")



compute_TSTR_Dp('realworld')
print('\n\n')
compute_TSTR_Dp('cwru')

Source class: WAL

Domain: 10
x_dp_dom.shape: (612, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Training and evaluating on Dp data via cross-validation...
	Epoch 10/50 - Train loss: 0.0329 - Val accuracy: 0.9898 - Val loss: 0.0490 - LR: 8.00e-05
	Epoch 20/50 - Train loss: 0.0102 - Val accuracy: 0.9796 - Val loss: 0.0500 - LR: 6.00e-05
	Epoch 30/50 - Train loss: 0.0076 - Val accuracy: 0.9898 - Val loss: 0.0413 - LR: 4.00e-05
	Epoch 40/50 - Train loss: 0.0050 - Val accuracy: 0.9796 - Val loss: 0.0450 - LR: 2.00e-05
	Epoch 50/50 - Train loss: 0.0052 - Val accuracy: 0.9898 - Val loss: 0.0430 - LR: 0.00e+00
	Best epoch: 33 - Best val accuracy: 0.9898 - Best val loss: 0.0271

	Epoch 10/50 - Train loss: 0.0369 - Val accuracy: 1.0000 - Val loss: 0.0262 - LR: 8.00e-05
	Epoch 20/50 - Train loss: 0.0164 - Val accuracy: 0.9898 - Val loss: 0.0169 - LR: 6.00e-05
	Epoch 30/50 - Train loss: 0.0047 - Val accuracy: 1.0000 - Val loss: 0.0126 - LR: 4.00e-05
	Epoch 40/50 - Train loss: 0.0056 - Val accuracy: 1.

In [ ]:
def compute_TSTR_Df(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        # Train on Df data
        print('Training on Df data...')
        df_model = train_only(x_df, y_df, dataset, num_epochs=50)

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Evaluate on Dp data
            acc, loss = evaluate_model(df_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, 'real', acc, loss, 'Df', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTR_Df('realworld')
print('\n\n')
compute_TSTR_Df('cwru')

Source class: WAL

x_df.shape: (5288, 3, 128) | np.unique(y_df): [1 2 3]

Training on Df data...
	Epoch 10/50 - Train loss: 0.0373 - Val accuracy: 0.9839 - Val loss: 0.0451 - LR: 8.00e-05
	Epoch 20/50 - Train loss: 0.0155 - Val accuracy: 0.9858 - Val loss: 0.0407 - LR: 6.00e-05
	Epoch 30/50 - Train loss: 0.0037 - Val accuracy: 0.9839 - Val loss: 0.0445 - LR: 4.00e-05
	Epoch 40/50 - Train loss: 0.0028 - Val accuracy: 0.9858 - Val loss: 0.0435 - LR: 2.00e-05
	Epoch 50/50 - Train loss: 0.0014 - Val accuracy: 0.9868 - Val loss: 0.0424 - LR: 0.00e+00
	Best epoch: 27 - Best val accuracy: 0.9858 - Best val loss: 0.0366

Domain: 10
x_dp_dom.shape: (612, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Source class: WAL | Domain: 10 | Accuracy: 0.9493 | Loss: 0.1271

Domain: 11
x_dp_dom.shape: (581, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Source class: WAL | Domain: 11 | Accuracy: 0.9639 | Loss: 0.1611

Domain: 12
x_dp_dom.shape: (577, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Source class: WAL | Domain: 

In [ ]:
def compute_TSTR_Syn(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load latent synthetic data
            x_syn_dom_lat, y_syn_dom_lat, k_syn_dom_lat = get_syn_data(dataset, src_class, domain, mode='lat')
            print(f'x_syn_dom_lat.shape: {x_syn_dom_lat.shape} | np.unique(y_syn_dom_lat): {np.unique(y_syn_dom_lat)}')

            # Load reference synthetic data
            x_syn_dom_ref, y_syn_dom_ref, k_syn_dom_ref = get_syn_data(dataset, src_class, domain, mode='ref')
            print(f'x_syn_dom_ref.shape: {x_syn_dom_ref.shape} | np.unique(y_syn_dom_ref): {np.unique(y_syn_dom_ref)}')

            # Join latent and reference synthetic data
            x_syn_dom_joint = np.concatenate([x_syn_dom_lat, x_syn_dom_ref], axis=0)
            y_syn_dom_joint = np.concatenate([y_syn_dom_lat, y_syn_dom_ref], axis=0)
            k_syn_dom_joint = np.concatenate([k_syn_dom_lat, k_syn_dom_ref], axis=0)
            print(f'x_syn_dom_joint.shape: {x_syn_dom_joint.shape} | np.unique(y_syn_dom_joint): {np.unique(y_syn_dom_joint)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on latent synthetic data and evaluate on Dp data
            print('Training on latent synthetic data...')
            acc_lat, loss_lat = train_and_test(x_syn_dom_lat, y_syn_dom_lat, x_dp_dom, y_dp_dom, dataset, num_epochs=50)
            save_scores(src_class, domain, 'lat', acc_lat, loss_lat, 'Syn', dataset)
            accs.append(acc_lat)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Latent): {acc_lat:.4f} | Loss: {loss_lat:.4f}\n')

            # Train on reference synthetic data and evaluate on Dp data
            print('Training on reference synthetic data...')
            acc_ref, loss_ref = train_and_test(x_syn_dom_ref, y_syn_dom_ref, x_dp_dom, y_dp_dom, dataset, num_epochs=50)
            save_scores(src_class, domain, 'ref', acc_ref, loss_ref, 'Syn', dataset)
            accs.append(acc_ref)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Reference): {acc_ref:.4f} | Loss: {loss_ref:.4f}\n')

            # Train on joint synthetic data and evaluate on Dp data
            print('Training on joint synthetic data...')
            acc_joint, loss_joint = train_and_test(x_syn_dom_joint, y_syn_dom_joint, x_dp_dom, y_dp_dom, dataset, num_epochs=50)
            save_scores(src_class, domain, 'joint', acc_joint, loss_joint, 'Syn', dataset)
            accs.append(acc_joint)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Joint): {acc_joint:.4f} | Loss: {loss_joint:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTR_Syn('realworld')
print('\n\n')
compute_TSTR_Syn('cwru')

Source class: WAL

Domain: 10
x_syn_dom_lat.shape: (741, 3, 128) | np.unique(y_syn_dom_lat): [1 2 3]
x_syn_dom_ref.shape: (741, 3, 128) | np.unique(y_syn_dom_ref): [1 2 3]
x_syn_dom_joint.shape: (1482, 3, 128) | np.unique(y_syn_dom_joint): [1 2 3]
x_dp_dom.shape: (612, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Training on latent synthetic data...
	Epoch 10/50 - Train loss: 0.0486 - Val accuracy: 0.9866 - Val loss: 0.0455 - LR: 8.00e-05
	Epoch 20/50 - Train loss: 0.0138 - Val accuracy: 0.9866 - Val loss: 0.0267 - LR: 6.00e-05
	Epoch 30/50 - Train loss: 0.0172 - Val accuracy: 0.9933 - Val loss: 0.0360 - LR: 4.00e-05
	Epoch 40/50 - Train loss: 0.0051 - Val accuracy: 0.9933 - Val loss: 0.0256 - LR: 2.00e-05
	Epoch 50/50 - Train loss: 0.0044 - Val accuracy: 0.9933 - Val loss: 0.0208 - LR: 0.00e+00
	Best epoch: 23 - Best val accuracy: 0.9933 - Best val loss: 0.0196

Source class: WAL | Domain: 10 | Accuracy (Latent): 0.8252 | Loss: 0.6434

Training on reference synthetic data...
	Epoch 10/50 -

In [ ]:
def compute_TSTR_Df_Syn(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        # Train on Df data
        print('Training on Df data...')
        df_model = train_only(x_df, y_df, dataset, num_epochs=50)

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load latent synthetic data
            x_syn_dom_lat, y_syn_dom_lat, k_syn_dom_lat = get_syn_data(dataset, src_class, domain, mode='lat')
            print(f'x_syn_dom_lat.shape: {x_syn_dom_lat.shape} | np.unique(y_syn_dom_lat): {np.unique(y_syn_dom_lat)}')

            # Load reference synthetic data
            x_syn_dom_ref, y_syn_dom_ref, k_syn_dom_ref = get_syn_data(dataset, src_class, domain, mode='ref')
            print(f'x_syn_dom_ref.shape: {x_syn_dom_ref.shape} | np.unique(y_syn_dom_ref): {np.unique(y_syn_dom_ref)}')

            # Join latent and reference synthetic data
            x_syn_dom_joint = np.concatenate([x_syn_dom_lat, x_syn_dom_ref], axis=0)
            y_syn_dom_joint = np.concatenate([y_syn_dom_lat, y_syn_dom_ref], axis=0)
            k_syn_dom_joint = np.concatenate([k_syn_dom_lat, k_syn_dom_ref], axis=0)
            print(f'x_syn_dom_joint.shape: {x_syn_dom_joint.shape} | np.unique(y_syn_dom_joint): {np.unique(y_syn_dom_joint)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Fine-tune Df model on latent synthetic data and evaluate on Dp data
            print('Fine-tuning on latent synthetic data...')
            df_lat_model = fine_tune(copy.deepcopy(df_model), x_syn_dom_lat, y_syn_dom_lat, num_epochs=50)
            acc_lat, loss_lat = evaluate_model(df_lat_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, 'lat', acc_lat, loss_lat, 'Df_Syn', dataset)
            accs.append(acc_lat)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Latent): {acc_lat:.4f} | Loss: {loss_lat:.4f}\n')

            # Fine-tune Df model on reference synthetic data and evaluate on Dp data
            print('Fine-tuning on reference synthetic data...')
            df_ref_model = fine_tune(copy.deepcopy(df_model), x_syn_dom_ref, y_syn_dom_ref, num_epochs=50)
            acc_ref, loss_ref = evaluate_model(df_ref_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, 'ref', acc_ref, loss_ref, 'Df_Syn', dataset)
            accs.append(acc_ref)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Reference): {acc_ref:.4f} | Loss: {loss_ref:.4f}\n')

            # Fine-tune Df model on joint synthetic data and evaluate on Dp data
            print('Fine-tuning on joint synthetic data...')
            df_joint_model = fine_tune(copy.deepcopy(df_model), x_syn_dom_joint, y_syn_dom_joint, num_epochs=50)
            acc_joint, loss_joint = evaluate_model(df_joint_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, 'joint', acc_joint, loss_joint, 'Df_Syn', dataset)
            accs.append(acc_joint)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Joint): {acc_joint:.4f} | Loss: {loss_joint:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTR_Df_Syn('realworld')
print('\n\n')
compute_TSTR_Df_Syn('cwru')

Source class: WAL

x_df.shape: (5288, 3, 128) | np.unique(y_df): [1 2 3]

Training on Df data...
	Epoch 10/50 - Train loss: 0.0332 - Val accuracy: 0.9811 - Val loss: 0.0506 - LR: 8.00e-05
	Epoch 20/50 - Train loss: 0.0096 - Val accuracy: 0.9839 - Val loss: 0.0480 - LR: 6.00e-05
	Epoch 30/50 - Train loss: 0.0040 - Val accuracy: 0.9839 - Val loss: 0.0517 - LR: 4.00e-05
	Epoch 40/50 - Train loss: 0.0040 - Val accuracy: 0.9858 - Val loss: 0.0487 - LR: 2.00e-05
	Epoch 50/50 - Train loss: 0.0017 - Val accuracy: 0.9849 - Val loss: 0.0550 - LR: 0.00e+00
	Best epoch: 27 - Best val accuracy: 0.9839 - Best val loss: 0.0478

Domain: 10
x_syn_dom_lat.shape: (741, 3, 128) | np.unique(y_syn_dom_lat): [1 2 3]
x_syn_dom_ref.shape: (741, 3, 128) | np.unique(y_syn_dom_ref): [1 2 3]
x_syn_dom_joint.shape: (1482, 3, 128) | np.unique(y_syn_dom_joint): [1 2 3]
x_dp_dom.shape: (612, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Fine-tuning on latent synthetic data...
	Epoch 10/50 - Train loss: 0.0872 - Val accuracy

In [ ]:
def compute_TSTRFS_Dpfs(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load Dpfs data
            x_dpfs_dom, y_dpfs_dom, k_dpfs_dom = get_data(dataset, 'dpfs', src_class, domain)
            print(f'x_dpfs_dom.shape: {x_dpfs_dom.shape} | np.unique(y_dpfs_dom): {np.unique(y_dpfs_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on Dpfs data and test on Dp data
            print('Training on Dpfs data...')
            acc, loss = train_and_test(x_dpfs_dom, y_dpfs_dom, x_dp_dom, y_dp_dom, dataset, num_epochs=50)
            save_scores(src_class, domain, 'real', acc, loss, 'FS_Dpfs', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")



compute_TSTRFS_Dpfs('realworld')
print('\n\n')
compute_TSTRFS_Dpfs('cwru')

In [ ]:
def compute_TSTRFS_Df_Dpfs(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        # Train on Df data
        print('Training on Df data...')
        df_model = train_only(x_df, y_df, dataset, num_epochs=50)

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load Dpfs data
            x_dpfs_dom, y_dpfs_dom, k_dpfs_dom = get_data(dataset, 'dpfs', src_class, domain)
            print(f'x_dpfs_dom.shape: {x_dpfs_dom.shape} | np.unique(y_dpfs_dom): {np.unique(y_dpfs_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}')

            # Fine-tune Df model on Dpfs data and test on Dp data
            print('Fine-tuning on Dpfs data...')
            df_dpfs_model = fine_tune(copy.deepcopy(df_model), x_dpfs_dom, y_dpfs_dom, num_epochs=50)
            acc, loss = evaluate_model(df_dpfs_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, 'real', acc, loss, 'FS_Df_Dpfs', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")



compute_TSTRFS_Df_Dpfs('realworld')
print('\n\n')
compute_TSTRFS_Df_Dpfs('cwru')

Source class: WAL

x_df.shape: (5288, 3, 128) | np.unique(y_df): [1 2 3]

Training on Df data...
	Epoch 10/50 - Train loss: 0.0376 - Val accuracy: 0.9783 - Val loss: 0.0468 - LR: 8.00e-05
	Epoch 20/50 - Train loss: 0.0093 - Val accuracy: 0.9802 - Val loss: 0.0420 - LR: 6.00e-05
	Epoch 30/50 - Train loss: 0.0044 - Val accuracy: 0.9783 - Val loss: 0.0481 - LR: 4.00e-05
	Epoch 40/50 - Train loss: 0.0021 - Val accuracy: 0.9802 - Val loss: 0.0471 - LR: 2.00e-05
	Epoch 50/50 - Train loss: 0.0019 - Val accuracy: 0.9820 - Val loss: 0.0493 - LR: 0.00e+00
	Best epoch: 16 - Best val accuracy: 0.9811 - Best val loss: 0.0403

Domain: 10
x_dpfs_dom.shape: (15, 3, 128) | np.unique(y_dpfs_dom): [1 2 3]
x_dp_dom.shape: (612, 3, 128) | np.unique(y_dp_dom): [1 2 3]
Fine-tuning on Dpfs data...
	Epoch 10/50 - Train loss: 0.9080 - Val accuracy: 1.0000 - Val loss: 0.0304 - LR: 8.00e-06
	Epoch 20/50 - Train loss: 0.4185 - Val accuracy: 1.0000 - Val loss: 0.0950 - LR: 6.00e-06
	Epoch 30/50 - Train loss: 0.3926

In [11]:
def compute_TSTRFS_Syn(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load latent synthetic data
            x_syn_dom_lat, y_syn_dom_lat, k_syn_dom_lat = get_syn_data(dataset, src_class, domain, mode='lat', fs=True)
            print(f'x_syn_dom_lat.shape: {x_syn_dom_lat.shape} | np.unique(y_syn_dom_lat): {np.unique(y_syn_dom_lat)}')

            # Load reference synthetic data
            x_syn_dom_ref, y_syn_dom_ref, k_syn_dom_ref = get_syn_data(dataset, src_class, domain, mode='ref', fs=True)
            print(f'x_syn_dom_ref.shape: {x_syn_dom_ref.shape} | np.unique(y_syn_dom_ref): {np.unique(y_syn_dom_ref)}')

            # Join latent and reference synthetic data
            x_syn_dom_joint = np.concatenate([x_syn_dom_lat, x_syn_dom_ref], axis=0)
            y_syn_dom_joint = np.concatenate([y_syn_dom_lat, y_syn_dom_ref], axis=0)
            k_syn_dom_joint = np.concatenate([k_syn_dom_lat, k_syn_dom_ref], axis=0)
            print(f'x_syn_dom_joint.shape: {x_syn_dom_joint.shape} | np.unique(y_syn_dom_joint): {np.unique(y_syn_dom_joint)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on latent synthetic data and evaluate on Dp data
            print('Training on latent synthetic data...')
            acc_lat, loss_lat = train_and_test(x_syn_dom_lat, y_syn_dom_lat, x_dp_dom, y_dp_dom, dataset, num_epochs=50)
            save_scores(src_class, domain, 'lat', acc_lat, loss_lat, 'FS_Syn', dataset)
            accs.append(acc_lat)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Latent): {acc_lat:.4f} | Loss: {loss_lat:.4f}\n')

            # Train on reference synthetic data and evaluate on Dp data
            print('Training on reference synthetic data...')
            acc_ref, loss_ref = train_and_test(x_syn_dom_ref, y_syn_dom_ref, x_dp_dom, y_dp_dom, dataset, num_epochs=50)
            save_scores(src_class, domain, 'ref', acc_ref, loss_ref, 'FS_Syn', dataset)
            accs.append(acc_ref)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Reference): {acc_ref:.4f} | Loss: {loss_ref:.4f}\n')

            # Train on joint synthetic data and evaluate on Dp data
            print('Training on joint synthetic data...')
            acc_joint, loss_joint = train_and_test(x_syn_dom_joint, y_syn_dom_joint, x_dp_dom, y_dp_dom, dataset, num_epochs=50)
            save_scores(src_class, domain, 'joint', acc_joint, loss_joint, 'FS_Syn', dataset)
            accs.append(acc_joint)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Joint): {acc_joint:.4f} | Loss: {loss_joint:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTRFS_Syn('realworld')
print('\n\n')
compute_TSTRFS_Syn('cwru')

Source class: WAL

Domain: 10
x_syn_dom_lat.shape: (741, 3, 128) | np.unique(y_syn_dom_lat): [1 2 3]
x_syn_dom_ref.shape: (741, 3, 128) | np.unique(y_syn_dom_ref): [1 2 3]
x_syn_dom_joint.shape: (1482, 3, 128) | np.unique(y_syn_dom_joint): [1 2 3]
x_dp_dom.shape: (612, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Training on latent synthetic data...
	Epoch 10/50 - Train loss: 0.0522 - Val accuracy: 0.9866 - Val loss: 0.0336 - LR: 8.00e-05
	Epoch 20/50 - Train loss: 0.0117 - Val accuracy: 0.9933 - Val loss: 0.0126 - LR: 6.00e-05
	Epoch 30/50 - Train loss: 0.0090 - Val accuracy: 0.9933 - Val loss: 0.0212 - LR: 4.00e-05
	Epoch 40/50 - Train loss: 0.0029 - Val accuracy: 0.9933 - Val loss: 0.0187 - LR: 2.00e-05
	Epoch 50/50 - Train loss: 0.0042 - Val accuracy: 0.9933 - Val loss: 0.0229 - LR: 0.00e+00
	Best epoch: 25 - Best val accuracy: 0.9933 - Best val loss: 0.0090

Source class: WAL | Domain: 10 | Accuracy (Latent): 0.9167 | Loss: 0.3861

Training on reference synthetic data...
	Epoch 10/50 -

In [12]:
def compute_TSTRFS_Df_Syn(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        # Train on Df data
        print('Training on Df data...')
        df_model = train_only(x_df, y_df, dataset, num_epochs=50)

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load latent synthetic data
            x_syn_dom_lat, y_syn_dom_lat, k_syn_dom_lat = get_syn_data(dataset, src_class, domain, mode='lat', fs=True)
            print(f'x_syn_dom_lat.shape: {x_syn_dom_lat.shape} | np.unique(y_syn_dom_lat): {np.unique(y_syn_dom_lat)}')

            # Load reference synthetic data
            x_syn_dom_ref, y_syn_dom_ref, k_syn_dom_ref = get_syn_data(dataset, src_class, domain, mode='ref', fs=True)
            print(f'x_syn_dom_ref.shape: {x_syn_dom_ref.shape} | np.unique(y_syn_dom_ref): {np.unique(y_syn_dom_ref)}')

            # Join latent and reference synthetic data
            x_syn_dom_joint = np.concatenate([x_syn_dom_lat, x_syn_dom_ref], axis=0)
            y_syn_dom_joint = np.concatenate([y_syn_dom_lat, y_syn_dom_ref], axis=0)
            k_syn_dom_joint = np.concatenate([k_syn_dom_lat, k_syn_dom_ref], axis=0)
            print(f'x_syn_dom_joint.shape: {x_syn_dom_joint.shape} | np.unique(y_syn_dom_joint): {np.unique(y_syn_dom_joint)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Fine-tune Df model on latent synthetic data and evaluate on Dp data
            print('Fine-tuning on latent synthetic data...')
            df_lat_model = fine_tune(copy.deepcopy(df_model), x_syn_dom_lat, y_syn_dom_lat, num_epochs=50)
            acc_lat, loss_lat = evaluate_model(df_lat_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, 'lat', acc_lat, loss_lat, 'FS_Df_Syn', dataset)
            accs.append(acc_lat)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Latent): {acc_lat:.4f} | Loss: {loss_lat:.4f}\n')

            # Fine-tune Df model on reference synthetic data and evaluate on Dp data
            print('Fine-tuning on reference synthetic data...')
            df_ref_model = fine_tune(copy.deepcopy(df_model), x_syn_dom_ref, y_syn_dom_ref, num_epochs=50)
            acc_ref, loss_ref = evaluate_model(df_ref_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, 'ref', acc_ref, loss_ref, 'FS_Df_Syn', dataset)
            accs.append(acc_ref)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Reference): {acc_ref:.4f} | Loss: {loss_ref:.4f}\n')

            # Fine-tune Df model on joint synthetic data and evaluate on Dp data
            print('Fine-tuning on joint synthetic data...')
            df_joint_model = fine_tune(copy.deepcopy(df_model), x_syn_dom_joint, y_syn_dom_joint, num_epochs=50)
            acc_joint, loss_joint = evaluate_model(df_joint_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, 'joint', acc_joint, loss_joint, 'FS_Df_Syn', dataset)
            accs.append(acc_joint)

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTRFS_Df_Syn('realworld')
print('\n\n')
compute_TSTRFS_Df_Syn('cwru')

Source class: WAL

x_df.shape: (5288, 3, 128) | np.unique(y_df): [1 2 3]

Training on Df data...
	Epoch 10/50 - Train loss: 0.0248 - Val accuracy: 0.9839 - Val loss: 0.0356 - LR: 8.00e-05
	Epoch 20/50 - Train loss: 0.0092 - Val accuracy: 0.9830 - Val loss: 0.0409 - LR: 6.00e-05
	Epoch 30/50 - Train loss: 0.0045 - Val accuracy: 0.9868 - Val loss: 0.0372 - LR: 4.00e-05
	Epoch 40/50 - Train loss: 0.0021 - Val accuracy: 0.9858 - Val loss: 0.0400 - LR: 2.00e-05
	Epoch 50/50 - Train loss: 0.0018 - Val accuracy: 0.9887 - Val loss: 0.0384 - LR: 0.00e+00
	Best epoch: 19 - Best val accuracy: 0.9887 - Best val loss: 0.0335

Domain: 10
x_syn_dom_lat.shape: (741, 3, 128) | np.unique(y_syn_dom_lat): [1 2 3]
x_syn_dom_ref.shape: (741, 3, 128) | np.unique(y_syn_dom_ref): [1 2 3]
x_syn_dom_joint.shape: (1482, 3, 128) | np.unique(y_syn_dom_joint): [1 2 3]
x_dp_dom.shape: (612, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Fine-tuning on latent synthetic data...
	Epoch 10/50 - Train loss: 0.1041 - Val accuracy

In [13]:
def compute_TSTRFS_Syn_Dpfs(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load latent synthetic data
            x_syn_dom_lat, y_syn_dom_lat, k_syn_dom_lat = get_syn_data(dataset, src_class, domain, mode='lat', fs=True)
            print(f'x_syn_dom_lat.shape: {x_syn_dom_lat.shape} | np.unique(y_syn_dom_lat): {np.unique(y_syn_dom_lat)}')

            # Load reference synthetic data
            x_syn_dom_ref, y_syn_dom_ref, k_syn_dom_ref = get_syn_data(dataset, src_class, domain, mode='ref', fs=True)
            print(f'x_syn_dom_ref.shape: {x_syn_dom_ref.shape} | np.unique(y_syn_dom_ref): {np.unique(y_syn_dom_ref)}')

            # Join latent and reference synthetic data
            x_syn_dom_joint = np.concatenate([x_syn_dom_lat, x_syn_dom_ref], axis=0)
            y_syn_dom_joint = np.concatenate([y_syn_dom_lat, y_syn_dom_ref], axis=0)
            k_syn_dom_joint = np.concatenate([k_syn_dom_lat, k_syn_dom_ref], axis=0)
            print(f'x_syn_dom_joint.shape: {x_syn_dom_joint.shape} | np.unique(y_syn_dom_joint): {np.unique(y_syn_dom_joint)}')

            # Load Dpfs data
            x_dpfs_dom, y_dpfs_dom, k_dpfs_dom = get_data(dataset, 'dpfs', src_class, domain)
            print(f'x_dpfs_dom.shape: {x_dpfs_dom.shape} | np.unique(y_dpfs_dom): {np.unique(y_dpfs_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on latent synthetic data
            print('Training on latent synthetic data...')
            lat_model = train_only(x_syn_dom_lat, y_syn_dom_lat, dataset, num_epochs=50)

            # Fine-tune latent model on Dpfs data and evaluate on Dp data
            print('Fine-tuning on Dpfs data...')
            lat_dpfs_model = fine_tune(copy.deepcopy(lat_model), x_dpfs_dom, y_dpfs_dom, num_epochs=50)
            acc_lat, loss_lat = evaluate_model(lat_dpfs_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, 'lat', acc_lat, loss_lat, 'FS_Syn_Dpfs', dataset)
            accs.append(acc_lat)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Latent): {acc_lat:.4f} | Loss: {loss_lat:.4f}\n')

            # Train on reference synthetic data
            print('Training on reference synthetic data...')
            ref_model = train_only(x_syn_dom_ref, y_syn_dom_ref, dataset, num_epochs=50)

            # Fine-tune reference model on Dpfs data and evaluate on Dp data
            print('Fine-tuning on Dpfs data...')
            ref_dpfs_model = fine_tune(copy.deepcopy(ref_model), x_dpfs_dom, y_dpfs_dom, num_epochs=50)
            acc_ref, loss_ref = evaluate_model(ref_dpfs_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, 'ref', acc_ref, loss_ref, 'FS_Syn_Dpfs', dataset)
            accs.append(acc_ref)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Reference): {acc_ref:.4f} | Loss: {loss_ref:.4f}\n')

            # Train on joint synthetic data
            print('Training on joint synthetic data...')
            joint_model = train_only(x_syn_dom_joint, y_syn_dom_joint, dataset, num_epochs=50)

            # Fine-tune joint model on Dpfs data and evaluate on Dp data
            print('Fine-tuning on Dpfs data...')
            joint_dpfs_model = fine_tune(copy.deepcopy(joint_model), x_dpfs_dom, y_dpfs_dom, num_epochs=50)
            acc_joint, loss_joint = evaluate_model(joint_dpfs_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, 'joint', acc_joint, loss_joint, 'FS_Syn_Dpfs', dataset)
            accs.append(acc_joint)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Joint): {acc_joint:.4f} | Loss: {loss_joint:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTRFS_Syn_Dpfs('realworld')
print('\n\n')
compute_TSTRFS_Syn_Dpfs('cwru')

Source class: WAL

Domain: 10
x_syn_dom_lat.shape: (741, 3, 128) | np.unique(y_syn_dom_lat): [1 2 3]
x_syn_dom_ref.shape: (741, 3, 128) | np.unique(y_syn_dom_ref): [1 2 3]
x_syn_dom_joint.shape: (1482, 3, 128) | np.unique(y_syn_dom_joint): [1 2 3]
x_dpfs_dom.shape: (15, 3, 128) | np.unique(y_dpfs_dom): [1 2 3]
x_dp_dom.shape: (612, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Training on latent synthetic data...
	Epoch 10/50 - Train loss: 0.0212 - Val accuracy: 1.0000 - Val loss: 0.0195 - LR: 8.00e-05
	Epoch 20/50 - Train loss: 0.0056 - Val accuracy: 0.9933 - Val loss: 0.0168 - LR: 6.00e-05
	Epoch 30/50 - Train loss: 0.0051 - Val accuracy: 0.9866 - Val loss: 0.0191 - LR: 4.00e-05
	Epoch 40/50 - Train loss: 0.0022 - Val accuracy: 0.9933 - Val loss: 0.0104 - LR: 2.00e-05
	Epoch 50/50 - Train loss: 0.0020 - Val accuracy: 0.9933 - Val loss: 0.0094 - LR: 0.00e+00
	Best epoch: 26 - Best val accuracy: 1.0000 - Best val loss: 0.0080

Fine-tuning on Dpfs data...
	Epoch 10/50 - Train loss: 1.0143 - V

In [14]:
def compute_TSTRFS_Df_Syn_Dpfs(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        # Train on Df data
        print('Training on Df data...')
        df_model = train_only(x_df, y_df, dataset, num_epochs=50)

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load latent synthetic data
            x_syn_dom_lat, y_syn_dom_lat, k_syn_dom_lat = get_syn_data(dataset, src_class, domain, mode='lat', fs=True)
            print(f'x_syn_dom_lat.shape: {x_syn_dom_lat.shape} | np.unique(y_syn_dom_lat): {np.unique(y_syn_dom_lat)}')

            # Load reference synthetic data
            x_syn_dom_ref, y_syn_dom_ref, k_syn_dom_ref = get_syn_data(dataset, src_class, domain, mode='ref', fs=True)
            print(f'x_syn_dom_ref.shape: {x_syn_dom_ref.shape} | np.unique(y_syn_dom_ref): {np.unique(y_syn_dom_ref)}')

            # Join latent and reference synthetic data
            x_syn_dom_joint = np.concatenate([x_syn_dom_lat, x_syn_dom_ref], axis=0)
            y_syn_dom_joint = np.concatenate([y_syn_dom_lat, y_syn_dom_ref], axis=0)
            k_syn_dom_joint = np.concatenate([k_syn_dom_lat, k_syn_dom_ref], axis=0)
            print(f'x_syn_dom_joint.shape: {x_syn_dom_joint.shape} | np.unique(y_syn_dom_joint): {np.unique(y_syn_dom_joint)}')

            # Load Dpfs data
            x_dpfs_dom, y_dpfs_dom, k_dpfs_dom = get_data(dataset, 'dpfs', src_class, domain)
            print(f'x_dpfs_dom.shape: {x_dpfs_dom.shape} | np.unique(y_dpfs_dom): {np.unique(y_dpfs_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Fine-tune Df model on latent synthetic data
            print('Fine-tuning on latent synthetic data...')
            df_lat_model = fine_tune(copy.deepcopy(df_model), x_syn_dom_lat, y_syn_dom_lat, num_epochs=50)

            # Fine-tune Df-latent model on Dpfs data and evaluate on Dp data
            print('Fine-tuning on Dpfs data...')
            df_lat_dpfs_model = fine_tune(copy.deepcopy(df_lat_model), x_dpfs_dom, y_dpfs_dom, num_epochs=50)
            acc_lat, loss_lat = evaluate_model(df_lat_dpfs_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, 'lat', acc_lat, loss_lat, 'FS_Df_Syn_Dpfs', dataset)
            accs.append(acc_lat)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Latent): {acc_lat:.4f} | Loss: {loss_lat:.4f}\n')

            # Fine-tune Df model on reference synthetic data
            print('Fine-tuning on reference synthetic data...')
            df_ref_model = fine_tune(copy.deepcopy(df_model), x_syn_dom_ref, y_syn_dom_ref, num_epochs=50)

            # Fine-tune Df-reference model on Dpfs data and evaluate on Dp data
            print('Fine-tuning on Dpfs data...')
            df_ref_dpfs_model = fine_tune(copy.deepcopy(df_ref_model), x_dpfs_dom, y_dpfs_dom, num_epochs=50)
            acc_ref, loss_ref = evaluate_model(df_ref_dpfs_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, 'ref', acc_ref, loss_ref, 'FS_Df_Syn_Dpfs', dataset)
            accs.append(acc_ref)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Reference): {acc_ref:.4f} | Loss: {loss_ref:.4f}\n')

            # Fine-tune Df model on joint synthetic data
            print('Fine-tuning on joint synthetic data...')
            df_joint_model = fine_tune(copy.deepcopy(df_model), x_syn_dom_joint, y_syn_dom_joint, num_epochs=50)

            # Fine-tune Df-joint model on Dpfs data and evaluate on Dp data
            print('Fine-tuning on Dpfs data...')
            df_joint_dpfs_model = fine_tune(copy.deepcopy(df_joint_model), x_dpfs_dom, y_dpfs_dom, num_epochs=50)
            acc_joint, loss_joint = evaluate_model(df_joint_dpfs_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, 'joint', acc_joint, loss_joint, 'FS_Df_Syn_Dpfs', dataset)
            accs.append(acc_joint)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy (Joint): {acc_joint:.4f} | Loss: {loss_joint:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTRFS_Df_Syn_Dpfs('realworld')
print('\n\n')
compute_TSTRFS_Df_Syn_Dpfs('cwru')

Source class: WAL

x_df.shape: (5288, 3, 128) | np.unique(y_df): [1 2 3]

Training on Df data...
	Epoch 10/50 - Train loss: 0.0253 - Val accuracy: 0.9802 - Val loss: 0.0481 - LR: 8.00e-05
	Epoch 20/50 - Train loss: 0.0087 - Val accuracy: 0.9820 - Val loss: 0.0390 - LR: 6.00e-05
	Epoch 30/50 - Train loss: 0.0022 - Val accuracy: 0.9811 - Val loss: 0.0461 - LR: 4.00e-05
	Epoch 40/50 - Train loss: 0.0022 - Val accuracy: 0.9839 - Val loss: 0.0427 - LR: 2.00e-05
	Epoch 50/50 - Train loss: 0.0013 - Val accuracy: 0.9811 - Val loss: 0.0499 - LR: 0.00e+00
	Best epoch: 14 - Best val accuracy: 0.9858 - Best val loss: 0.0370

Domain: 10
x_syn_dom_lat.shape: (741, 3, 128) | np.unique(y_syn_dom_lat): [1 2 3]
x_syn_dom_ref.shape: (741, 3, 128) | np.unique(y_syn_dom_ref): [1 2 3]
x_syn_dom_joint.shape: (1482, 3, 128) | np.unique(y_syn_dom_joint): [1 2 3]
x_dpfs_dom.shape: (15, 3, 128) | np.unique(y_dpfs_dom): [1 2 3]
x_dp_dom.shape: (612, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Fine-tuning on latent sy